In [132]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [133]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [134]:
## Preprocessing the data
## Drop the irrelevent columns

df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [135]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [136]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [137]:
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])

In [138]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

In [139]:
geo_encoder = ohe.fit_transform(df[['Geography']])

In [140]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [141]:
geo_encoderdf = pd.DataFrame(geo_encoder,columns=ohe.get_feature_names_out(['Geography']))

In [142]:
geo_encoderdf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [143]:
## Combine the one hot encoder columns with the original data

df = pd.concat([df.drop('Geography',axis=1),geo_encoderdf],axis=1)

In [144]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [145]:
## save the encoder and scaler

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le_gender, file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe, file)

In [146]:
## Divide the dataset into indepent and dependent features

X = df.drop('Exited',axis=1)
y = df['Exited']


In [147]:
# split the data in training data 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# scale the feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [148]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# ANN Implementation

In [149]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [150]:
## Build our ANN Model

model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1,activation='sigmoid')

])

In [151]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [152]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [153]:
# compiel the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [154]:
## set up the Tensorflow

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [155]:
## Set up Early Stopping

Early_Stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [156]:
## Train the model

history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks = [tensorflow_callback, Early_Stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3865 - accuracy: 0.8403 - val_loss: 0.3410 - val_accuracy: 0.8640
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3572 - accuracy: 0.8572 - val_loss: 0.3482 - val_accuracy: 0.8535
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3476 - accuracy: 0.8572 - val_loss: 0.3485 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3457 - accuracy: 0.8576 - val_loss: 0.3543 - val_accuracy: 0.8515
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3414 - accuracy: 0.8608 - val_loss: 0.3488 - val_accuracy: 0.8635
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3423 - accuracy: 0.8565 - val_loss: 0.3389 - val_accuracy: 0.8585
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3350 - accuracy: 0.8616 - val_loss: 0.3474 - val_accuracy: 0.8595

In [157]:
model.save('model.h5')

d:\Conda\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [158]:
## Load the Tensorboard Extensoin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [161]:
%tensorboard --logdir log/fit20250802-083426

Reusing TensorBoard on port 6012 (pid 24844), started 0:00:03 ago. (Use '!kill 24844' to kill it.)